In [12]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import itertools

In [3]:
train_path="images/train"
validation_path="images/validation"
labels=["dog","cat"] #change this to our lables

In [4]:
train_batches=ImageDataGenerator().flow_from_directory(train_path,target_size=(224,224),classes=labels,batch_size=10)
validation_batches=ImageDataGenerator().flow_from_directory(validation_path,target_size=(224,224),classes=labels,batch_size=10)

Found 70 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [31]:
vgg16_model=keras.applications.mobilenet.MobileNet()

17227776/17225924 [==============================] - 68s 4us/step


In [32]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [33]:
#vgg model is not sequential so we need to convert it
model=Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

In [34]:
#we should remove the last dence layer of vgg_model to make it fit to our ourpose
model.layers.pop()

In [35]:
for layer in model.layers:
    layer.trainable=False

In [36]:
model.add(Dense(2,activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [37]:
model.compile(Adam(lr=.0005),loss="categorical_crossentropy",metrics=['accuracy'])
model.fit_generator(train_batches,steps_per_epoch=10,validation_data=validation_batches,validation_steps=4,epochs=30,verbose=2)

Epoch 1/1
 - 29s - loss: 0.6965 - acc: 0.4000 - val_loss: 0.6936 - val_acc: 0.5000


In [38]:
model.save("Landmark_model.h5")